In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.regression import *
import numpy as np
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet

import warnings
import holidays

warnings.filterwarnings(action='ignore') 

In [2]:
# path = f'./processed_data/train_merge.csv'
# df = pd.read_csv(path)
# df = df[df['item']=='BC']
# df['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in df['timestamp']]
# df = pd.get_dummies(df, columns=qual_col, prefix=qual_col)
# # df[~df["Date"].isin(kr_holidays)]


# name_lst = df.corr()['price(원/kg)'].sort_values().keys().to_list()
# value_lst = df.corr()['price(원/kg)'].sort_values().values

# print([a for a, b in zip(name_lst, value_lst) if abs(b)>0.15 ])

In [3]:
# 필요 정보
# item_lst : item 리스트 항목
# qual_col : one-hot-encoder 변환이 필요한 항목

item_lst = ['TG', 'CR', 'CB', 'RD', 'BC']
qual_col = ['corporation', 'location', 'holiday']
columns_dict_train = {
    'TG' : list(set(['Date', 'year', 'month', 'week', 'day', 'supply(kg)_y', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation','price(원/kg)'])),
    'CR' : list(set(['Date', 'year', 'month', 'week', 'day', 'location',   'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'CB' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', '무역수지', '수출 금액', '수출 중량', 'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'RD' : list(set(['Date', 'year', 'month', 'week', 'day', 'corporation', 'location', 'corporation', 'location', 'supply(kg)_y','price(원/kg)'])),
    'BC' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', 'corporation', 'corporation', 'corporation', 'location', 'supply(kg)_y','price(원/kg)']))
}

columns_dict_test= {
    'TG' : list(set(['Date', 'year', 'month', 'week', 'day', 'supply(kg)', '수출 금액', '무역수지', '전체 거래량', '수출 중량', 'corporation'])),
    'CR' : list(set(['Date', 'year', 'month', 'week', 'day', 'location',   'corporation', 'location', 'supply(kg)'])),
    'CB' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', '무역수지', '수출 금액', '수출 중량', 'corporation', 'location', 'supply(kg)'])),
    'RD' : list(set(['Date', 'year', 'month', 'week', 'day', 'corporation', 'location', 'corporation', 'location', 'supply(kg)'])),
    'BC' : list(set(['Date', 'year', 'month', 'week', 'day', 'location', 'corporation', 'corporation', 'corporation', 'location', 'supply(kg)']))
}

# item 선택
item = item_lst[0]
path = f'./processed_data/{item}_train_merge.csv'

# 한국 2019~2023 공휴일 정보
kr_holidays = holidays.KR(years=[2019,2020,2021,2022,2023])
#kr_holidays = [str(i) for i in kr_holidays]
kr_holidays = {str(i):kr_holidays[i] for i in kr_holidays}

In [4]:
predictions = []

test = pd.read_csv('./processed_data/test_merge.csv')

for i in range(5):
    item = item_lst[i]
    path = f'./processed_data/{item}_train_merge.csv'

    train = pd.read_csv(path)
    test_df = test[test['item']==item]
    
    # 데이터 전처리 프로세스
    #train = train[~train["Date"].isin(kr_holidays)] # Holyday 제거
    train['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in train['timestamp']]
    test_df['holiday'] = [kr_holidays[i] if i in list(kr_holidays.keys()) else 'None' for i in test_df['timestamp']]
    
    for l in qual_col:
        le = LabelEncoder()
        train[l]=le.fit_transform(train[l])
        test_df[l]=le.transform(test_df[l]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

    train = pd.get_dummies(train, columns=['day_name'], prefix=['day_name'])
    test_df = pd.get_dummies(test_df, columns=['day_name'], prefix=['day_name'])

    train = train[columns_dict_train[item]]
    train = train.rename(columns={'supply(kg)_y':'supply(kg)'})
    # train = train.rename(columns={'Date':'ds'})
    # train = train.rename(columns={'price(원/kg)':'y'})
    test_df = test_df[columns_dict_test[item]]

    # m = Prophet(seasonality_mode='additive')
    # m.fit(train)
    
    # test_df.rename(columns={'Date':'ds'},inplace=True)
    # forecast = m.predict(test_df)
    # print(forecast)
    # predictions.extend(forecast)

    s = setup(train, target = 'price(원/kg)', 
        session_id = 123, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'minmax',
        train_size=0.7)

    reg = create_model('rf')
    #reg = compare_models()
    tuned_reg = tune_model(reg, optimize = 'RMSE', fold=10, n_iter=50)
    prediction = predict_model(tuned_reg, data = test_df)
    predictions.extend(prediction['prediction_label'])

,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(15230, 12)"
4,Transformed data shape,"(15230, 12)"
5,Transformed train set shape,"(10661, 12)"
6,Transformed test set shape,"(4569, 12)"
7,Numeric features,10
8,Categorical features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1200.4028,4168748.4787,2041.7513,0.4646,2.1037,0.2871
1,1161.5155,4509777.5255,2123.6237,0.4250,2.2982,0.2692
2,1179.6406,3947286.5302,1986.7779,0.4716,2.3271,0.2829
3,1080.2573,3433946.1775,1853.0910,0.5239,2.3121,0.2477
4,1139.1869,3956598.4585,1989.1200,0.4686,2.1796,0.2692
5,1200.8659,4257860.0769,2063.4583,0.4666,2.4677,0.2660
6,1142.7639,4040146.6456,2010.0116,0.4912,2.1857,0.2674
7,1105.5484,3763439.8272,1939.9587,0.5310,2.0600,0.2660
8,1146.8974,4059000.4405,2014.6961,0.5157,2.0107,0.2703


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1059.7322,3229647.5871,1797.1220,0.5852,2.8949,0.2251
1,1056.3458,3510935.3592,1873.7490,0.5524,3.3022,0.2184
2,1071.0747,3210908.8661,1791.9009,0.5702,3.2103,0.2257
3,1000.7784,2808764.8482,1675.9370,0.6105,3.1015,0.2038
4,1017.4898,3042470.1398,1744.2678,0.5914,3.0824,0.2114
5,1088.2246,3277487.6839,1810.3833,0.5894,3.2956,0.2129
6,1070.1144,3327188.2306,1824.0582,0.5810,3.1130,0.2268
7,1027.0573,3262426.3180,1806.2188,0.5934,2.9932,0.2116
8,1015.5375,3012476.0870,1735.6486,0.6406,2.9480,0.2167


Fitting 10 folds for each of 50 candidates, totalling 500 fits


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(10661, 9)"
4,Transformed data shape,"(10661, 9)"
5,Transformed train set shape,"(7462, 9)"
6,Transformed test set shape,"(3199, 9)"
7,Numeric features,7
8,Categorical features,1
9,Preprocess,True


...

Fitting 10 folds for each of 50 candidates, totalling 500 fits


,Description,Value
0,Session id,123
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(13707, 9)"
4,Transformed data shape,"(13707, 9)"
5,Transformed train set shape,"(9594, 9)"
6,Transformed test set shape,"(4113, 9)"
7,Numeric features,7
8,Categorical features,1
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,287.5311,436011.6609,660.3118,0.7881,2.1420,0.2236
1,274.1824,449914.4097,670.7566,0.7586,2.1913,0.2084
2,312.4544,569809.6532,754.8574,0.7359,2.2671,0.2089
3,283.4060,450881.1075,671.4768,0.7687,2.0211,0.2304
4,313.6342,630863.1141,794.2689,0.7129,2.0353,0.2249
5,277.0520,429228.3255,655.1552,0.7778,2.1439,0.2429
6,272.8973,451493.8901,671.9330,0.7661,2.0797,0.2146
7,294.6696,527557.3183,726.3314,0.7493,2.0300,0.2228
8,278.9770,461298.8506,679.1898,0.7799,1.9214,0.2154


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,313.8472,438163.7144,661.9394,0.7870,2.6872,0.2186
1,302.5949,414444.9664,643.7740,0.7776,2.8029,0.2075
2,328.0624,530094.7320,728.0760,0.7543,2.7065,0.2006
3,298.5732,403944.4709,635.5663,0.7928,2.6690,0.2166
4,330.3856,581318.3077,762.4423,0.7354,2.6091,0.2213
5,308.9400,439688.2850,663.0900,0.7724,2.8552,0.2405
6,301.1203,439646.7574,663.0586,0.7722,2.6720,0.2143
7,314.5687,468234.7346,684.2768,0.7775,2.6233,0.2248
8,300.0294,434058.9867,658.8315,0.7929,2.5901,0.2063


Fitting 10 folds for each of 50 candidates, totalling 500 fits


In [5]:
result = pd.read_csv('./data/test.csv')
result['answer'] = predictions

del result['timestamp']
del result['item']
del result['corporation']
del result['location']

for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0


In [9]:
result.to_csv("reselt.csv", index=False)

In [7]:
train

,month,week,supply(kg),year,day,Date,price(원/kg),corporation,location
0,1,1.0,695871.0,2019,1.0,2019-01-01,0.0,0,0
1,1,1.0,695871.0,2019,2.0,2019-01-02,0.0,0,0
2,1,1.0,695871.0,2019,3.0,2019-01-03,2559.0,0,0
3,1,1.0,695871.0,2019,4.0,2019-01-04,2425.0,0,0
4,1,1.0,695871.0,2019,5.0,2019-01-05,2097.0,0,0
...,...,...,...,...,...,...,...,...,...
13702,3,9.0,61472.0,2023,2.0,2023-03-02,3144.0,4,0
13703,3,9.0,61472.0,2023,3.0,2023-03-03,3045.0,4,0
13704,3,9.0,61472.0,2023,1.0,2023-03-01,3816.0,4,1
13705,3,9.0,61472.0,2023,2.0,2023-03-02,3321.0,4,1


In [8]:
test_df

,month,supply(kg),week,year,day,Date,corporation,location
840,3,61472.0,9.0,2023,4.0,2023-03-04,0,0
841,3,61472.0,9.0,2023,5.0,2023-03-05,0,0
842,3,61472.0,10.0,2023,6.0,2023-03-06,0,0
843,3,61472.0,10.0,2023,7.0,2023-03-07,0,0
844,3,61472.0,10.0,2023,8.0,2023-03-08,0,0
...,...,...,...,...,...,...,...,...
1087,3,61472.0,13.0,2023,27.0,2023-03-27,4,1
1088,3,61472.0,13.0,2023,28.0,2023-03-28,4,1
1089,3,61472.0,13.0,2023,29.0,2023-03-29,4,1
1090,3,61472.0,13.0,2023,30.0,2023-03-30,4,1
